In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer, f1_score
from collections import Counter
import scipy.stats as stats # for the breakpoints in SAX
from scipy.stats import norm
from Symbol import SYMBOLS
from distances import MINDIST
from SAX_transf import SAX_transform
import warnings 
from dtw import dtw
from sklearn.model_selection import GridSearchCV
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.utils import to_time_series_dataset
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc


warnings.filterwarnings('ignore')
import os
import aeon
from aeon.datasets import load_classification

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [4]:
train_path = os.path.join(os.getcwd(), "datasets/classification/ECG200/ECG200_TRAIN.ts")
test_path = os.path.join(os.getcwd(), "datasets/classification/ECG200/ECG200_TEST.ts")

X_train = pd.read_csv(train_path, 
                      sep=",", 
                      header=None
                      )

X_train.columns = list(X_train.columns[:-1]) + ['label']
x_train, y_train = X_train.iloc[:, :-1], X_train["label"]

y_train = y_train.replace(-1, 0)

X_test = pd.read_csv(test_path, 
                      sep=",", 
                      header=None
                      )

X_test.columns = list(X_test.columns[:-1]) + ['label']
x_test, y_test = X_test.iloc[:, :-1], X_test["label"]

y_test = y_test.replace(-1, 0)

In [5]:
ts_1 = x_train.iloc[0,:] # first time series, length 96
num_seg = 10
alphabet_size = 5
sax = SAX_transform(ts_1, num_seg, alphabet_size)


In [7]:
sax.tsax_paa(ts_1)

[[1.3363511566666668,
  3.910320042070083,
  -19.682626457665805,
  3.8142034509822293],
 [-0.23096119222222222,
  7.936819341988922,
  0.189063253254119,
  2.4197842537011307],
 [-0.8800282388888889,
  8.803867186218554,
  -4.062270737902821,
  -0.9182103926810493],
 [-1.5219079333333334,
  -25.93118199827385,
  11.664236608181193,
  0.41919709087701523],
 [-1.5853730222222222,
  6.603912671199026,
  -4.2224166034640405,
  7.124885467674546],
 [-0.25865943444444445,
  2.5924567321834218,
  -5.7151537706823206,
  7.995239414256104],
 [0.5679134477777779, -16.07167893840013, 6.639161121825332, 6.22738791305367],
 [0.6310565600000001,
  -3.571906848417529,
  6.706575698152518,
  2.5189352415983572],
 [0.6952532066666666,
  1.6800929885966127,
  -1.4058630544378161,
  1.1368862031095655],
 [0.7478132839999999,
  -16.81803147865214,
  12.191846738006447,
  -1.2641244796340467]]

In [9]:
len(sax.calculate_tsax(10))

40